In [1]:
# prompt: connect to gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

In [3]:
from scipy.cluster.hierarchy import dendrogram
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(cluster_model, annotation_fontsize=4, **kwargs):
    # Create linkage matrix
    counts = np.zeros(cluster_model.children_.shape[0])
    n_samples = len(cluster_model.labels_)
    for i, merge in enumerate(cluster_model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [cluster_model.children_, cluster_model.distances_, counts]
    ).astype(float)

    # Plot dendrogram and capture its return value
    ddata = dendrogram(linkage_matrix, **kwargs)
    ax = plt.gca()
    # Adjust font size of distance annotations
    ax.tick_params(axis='x', which='major', labelsize=8)

    return ddata
def cluster_dendro(weights):
    # setting distance_threshold=0 ensures we compute the full tree.
    cluster_model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

    cluster_model = cluster_model.fit(weights)
    plt.title("Hierarchical Clustering Dendrogram")
    # plot the top three levels of the dendrogram
    plot_dendrogram(cluster_model, truncate_mode="level", p=3)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()


In [4]:
def normalize_blocks(data, block_size=41, epsilon=1e-8):
        normalized_data = data.copy()  # To avoid modifying the original array
        for i in range(0, normalized_data.shape[0], block_size):
            block = normalized_data[i:i+block_size]
            # Compute mean and std per feature over samples (axis 0) and time steps (axis 2)
            mean = block.mean(axis=(0,2), keepdims=True)
            std = block.std(axis=(0, 2), keepdims=True)
            # Normalize the block and handle division by zero by adding epsilon
            normalized_block = (block - mean) / (std + epsilon)
            normalized_data[i:i+block_size] = normalized_block
        return normalized_data


**Note: Genre labels -  Metal:0,   Classical: 1, Rock: 2, Blues: 3, Country: 4, Disco: 5, Hiphop: 6, Reggae: 7, Pop:8, Jazz:9**

In [9]:
# Clipped out first two observations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, GRU, Input
import numpy as np
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix
import random
import tensorflow as tf
import seaborn as sns
meta_accuracy=[]
for meta_index in range(5):
  accuracies = []


  for subject_index in range(5):
        trainX = np.load(f'/content/drive/MyDrive/Owned_files_to_transfer/SOCR_fmri_spatiotemporal_feature/trainX/trainX_subj{subject_index+1}.npy')

      # Assuming trainX and testX are NumPy arrays of shapes (492, 2105, 10) and (246, 2105, 10)
        trainX = normalize_blocks(trainX)
        trainX = np.transpose(trainX,(0,2,1))

        print(trainX.shape)
        testX = np.load(f'/content/drive/MyDrive/Owned_files_to_transfer/SOCR_fmri_spatiotemporal_feature/testX/testX_subj{subject_index+1}.npy')
        testX = normalize_blocks(testX)
        testX = np.transpose(testX,(0,2,1))
        print(testX.shape)
        trainY = np.load(f'/content/drive/MyDrive/Owned_files_to_transfer/SOCR_fmri_spatiotemporal_feature/trainY/trainY_subj{subject_index+1}.npy')
        print(trainY.shape)
        testY = np.load(f'/content/drive/MyDrive/Owned_files_to_transfer/SOCR_fmri_spatiotemporal_feature/testY/testY_subj{subject_index+1}.npy')
        print(testY.shape)

        model = Sequential()
        model.add(Input(shape=(7,2105)))
        #model.add(TimeDistributed(Dense(64, kernel_regularizer=regularizers.l2(0.01))))
        model.add(TimeDistributed(Dense(128, kernel_regularizer=regularizers.l2(0.01))))

        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=regularizers.l2(0.01)))  # Input shape (timesteps, features)
        model.add(Dropout(0.5))                      # Regularization
        model.add(Dense(10, activation='softmax'))   # Output layer for 10 classes
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        model.summary()
        history = model.fit(trainX[:,3:,:], trainY,
                            epochs=20,
                            batch_size=32,
                            validation_split=0.2,verbose=0)  # Using 20% of training data for validation
        test_loss, test_acc = model.evaluate(testX[:,3:,:], testY)
        print(f'Test accuracy: {test_acc:.4f}')
        accuracies.append(test_acc)
  print(f"The mean accuracy is {np.mean(accuracies)}")
  print(f"The standard deviation is {np.std(accuracies)}")
  meta_accuracy.append(np.mean(accuracies))
print(f"The meta mean accuracy across five run is {np.mean(meta_accuracy)}")
print(f"The standard deviation across five run is {np.std(meta_accuracy)}")

(492, 10, 2105)
(246, 10, 2105)
(492,)
(246,)


Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_33             │ (None, 7, 128)         │       269,568 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_33 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 402,442 (1.54 MB)

 Trainable params: 402,442 (1.54 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_57             │ (None, 7, 128)         │       269,568 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_57 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_115 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 402,442 (1.54 MB)

 Trainable params: 402,442 (1.54 MB)

 Non-trainable params: 0 (0.00 B)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3111 - loss: 3.4175
Test accuracy: 0.3252
The mean accuracy is 0.2910569101572037
The standard deviation is 0.036048457774104974
The meta mean accuracy across five run is 0.2845528471469879
The standard deviation across five run is 0.006148834452206904
